In [3]:
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.datasets import mnist
import numpy as np

# all gpu should be used
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 2 Logical GPUs


In [4]:
# Check tensorflow device
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13380349820003918595
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23365222400
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6130650749948279520
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23365222400
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16934262958898462278
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:03:00.0, compute capability: 8.9"
xla_global_id: 2144165316
]


2024-04-22 23:13:55.900397: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 23:13:55.900543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 23:13:55.900629: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 23:13:55.900698: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 23:13:55.900765: I tensorflow/compiler/xla/stream_executo

In [5]:
# hipermarametros

batch_size = 100
original_dim = 784
latent_dim = 2
intermediate_dim = 256
epochs = 100
epsilon_std = 0.7

In [6]:
# create incoder

x = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation="relu")(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)
z = Lambda(
    lambda args: args[0]
    + K.random_normal(
        shape=(K.shape(args[0])[0], latent_dim), mean=0.0, stddev=epsilon_std
    )
    * K.exp(args[1] / 2),
    output_shape=(latent_dim,),
)([z_mean, z_log_var])

encoder = Model(x, [z_mean, z_log_var, z], name="encoder")

In [7]:
# create sampling function


def sampling(args: tuple):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim), mean=0.0, stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [8]:
# create decoder

input_decoder = Input(shape=(latent_dim,), name="decoder_input")

decoder_h = Dense(intermediate_dim, activation="relu", name="decoder_h")(input_decoder)

x_decoded = Dense(original_dim, activation="sigmoid", name="flat_decoded")(decoder_h)

decoder = Model(input_decoder, x_decoded, name="decoder")

In [9]:
# create vae
output_combined = decoder(encoder(x)[2])
vae = Model(x, output_combined)
vae.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 encoder (Functional)        [(None, 2),               201988    
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
 decoder (Functional)        (None, 784)               202256    
                                                                 
Total params: 404,244
Trainable params: 404,244
Non-trainable params: 0
_________________________________________________________________


In [10]:
# loss function
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

vae.add_loss(K.mean(kl_loss) / original_dim)
vae.compile(optimizer="rmsprop", loss="binary_crossentropy")

In [11]:
# dataloads

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [12]:
# train
vae.fit(
    x_train,
    x_train,
    shuffle=True,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(x_test, x_test),
)

Epoch 1/100


2024-04-22 23:13:56.633711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-04-22 23:13:56.646845: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fdd869a7b60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-22 23:13:56.646861: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-04-22 23:13:56.646864: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-04-22 23:13:56.649033: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-22 23:13:56.674685: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver fo

600/600 [==============================] - 1s 2ms/step - loss: 0.2863 - val_loss: 0.2485
Epoch 2/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2447 - val_loss: 0.2406
Epoch 3/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2387 - val_loss: 0.2360
Epoch 4/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2351 - val_loss: 0.2328
Epoch 5/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2324 - val_loss: 0.2304
Epoch 6/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2301 - val_loss: 0.2282
Epoch 7/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2279 - val_loss: 0.2262
Epoch 8/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2260 - val_loss: 0.2243
Epoch 9/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2243 - val_loss: 0.2227
Epoch 10/100
600/600 [==============================] - 1s 1ms/step - loss: 0.2228 - val_loss: 0.2215
E

In [ ]:
# show results
import matplotlib.pyplot as plt

n = 15
digit_size = 28
figure = np.zeros((digit_size * n, digit_size * n))

grid_x = np.linspace(-15, 15, n)
grid_y = np.linspace(-15, 15, n)

for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[
            i * digit_size : (i + 1) * digit_size, j * digit_size : (j + 1) * digit_size
        ] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()